# Imports

In [31]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import clear_output
from tqdm import tqdm

In [32]:
import statsmodels.api as sm

import statsmodels.formula.api as smf
import pandas as pd

import geopandas as gpd
import matplotlib.pyplot as plt
import contextily
from shapely.geometry import Point, Polygon

In [33]:
import os
import glob
import pandas as pd

# Age and Gender

In [34]:
cd /Users/anurag/Documents/Paris_Joint_Project/

/Users/anurag/Documents/Paris_Joint_Project


In [35]:
all_filenames = ['Données Biody PC1.csv','Données Biody PC2.csv', 'Données Biody PC3.csv', 
                'Données Biody PC4.csv', 'Données Biody PC5.csv']
pData = pd.concat([pd.read_csv(f, sep = ';',  encoding ='latin-1')for f in all_filenames])
pData['unique_id'] = pData['Adresse'].astype(str)+str('-')+pData['Ville']
pData = pData.drop_duplicates(subset='unique_id', keep="first")

In [36]:
pData.head()

,Nom,Prénom,Sexe,Né(e) le,age,Adresse,Code postal,Ville,État/Province,Email,...,Eau extracellulaire,Taux d'hydratation hors graisse,IBMI,Indice masse grasse,Indice masse musculaire,Indice masse osseuse,Indice charge en muscle,Indice charge totale,Unnamed: 0,unique_id
0,Philippart,Odile,2,15/06/1953,66.72,13202,NaN,B,NaN,NaN,...,11.66,75.21,0.87,0.94,0.92,0.94,0.98,0.99,NaN,13202-B
1,Sinsi,Laurence,2,15/12/1972,47.22,15412,NaN,B,NaN,NaN,...,12.37,73.99,0.89,1.12,0.88,0.94,0.93,1.00,NaN,15412-B
2,Galice,Nicolas,1,15/01/1973,47.13,13507,NaN,A,NaN,NaN,...,21.25,74.91,1.17,0.99,1.26,1.43,0.87,0.89,NaN,13507-A
3,Prot,Odile,2,15/09/1955,64.47,4202,NaN,B,NaN,NaN,...,12.91,73.22,0.88,1.05,0.95,0.96,0.99,1.03,NaN,4202-B
4,Charavel,Pierre,1,15/10/1957,62.38,14842,NaN,A,NaN,NaN,...,17.27,73.05,1.18,1.40,1.18,1.18,1.00,1.17,NaN,14842-A


# NDVI

In [37]:
cd /Users/anurag/Documents/Paris_Joint_Project/

/Users/anurag/Documents/Paris_Joint_Project


In [38]:
NDVI = pd.read_csv("NDVI_100.csv")
NDVI = NDVI.drop(columns=['imageID','timeMillis','.geo'])
NDVI['system:index'] = NDVI['system:index'].str.replace('_', '/').apply(lambda x: x[:-2])
NDVI['system:index'] = pd.to_datetime(NDVI['system:index'], format='%Y/%m/%d')    
NDVI = NDVI.set_index('system:index').resample('D').ffill().reset_index()
NDVI_processed = NDVI.melt(id_vars = 'system:index')
NDVI_processed[[' LONGITUDE',' LATITUDE']]= NDVI_processed['variable'].str.split('_', expand = True).apply(lambda row : row.astype(float)/1000)
NDVI_processed.rename(columns={'system:index': ' UTC DATE'}, inplace=True)

In [39]:
NDVI_processed.head()

,UTC DATE,variable,value,LONGITUDE,LATITUDE
0,2018-05-09,2252_48817,5163.0,2.252,48.817
1,2018-05-10,2252_48817,5163.0,2.252,48.817
2,2018-05-11,2252_48817,5163.0,2.252,48.817
3,2018-05-12,2252_48817,5163.0,2.252,48.817
4,2018-05-13,2252_48817,5163.0,2.252,48.817


# MVPA

In [21]:
cd /Users/anurag/Documents/Paris_Joint_Project/

/Users/anurag/Documents/Paris_Joint_Project


In [22]:
# done while combining itself
#CALCULATIONS ARE AS FOLLOWS

# walking speed is between 5.0 km/h and 9 km/hr 
# averge running speed is 12 km/hr, highest is 44.7397632 kmph
# average cycling speed is 15.5 km/hr
# rough initial estimate keep 20 km/hr

# phy_act_coordinates = phy_act_coordinates[phy_act_coordinates[' SPEED']<20]

# phy_act_coordinates = coordinates[(coordinates[' SPEED']>5) & (coordinates[' SPEED']<20)]
# still_coordinates = coordinates[(coordinates[' SPEED']<1)]

#Built Evironment Factors

In [67]:
cd /Users/anurag/Documents/Paris_Joint_Project/

/Users/anurag/Documents/Paris_Joint_Project


In [68]:
trees = pd.read_csv('trees.csv')
transitStops = pd.read_csv('trainStations.csv')

# Combine All

In [69]:
cd /Users/anurag/Documents/Paris_Joint_Project/common

/Users/anurag/Documents/Paris_Joint_Project/common


In [70]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [79]:
def preprocessing(filename):

    try:
        if filename in ['2018-05-19_66_np6_5G_10685-A.csv',
        '2019-02-07_105_np86_1NM_2399-B.csv',
        '2018-06-15_119_np12_13N_16570-A.csv']:
            df = pd.read_csv(filename, sep = ';',  encoding = "utf-8")
        else:
            df = pd.read_csv(filename, sep = ',', encoding = "utf-8")
          
        df['file_name']=filename
        df['unique_id'] = filename.split('_')[-1].split('.')[0] 

        # Resample to higher time scale
        resampled = df.iloc[::12, :] # sample every minute. 


        # Add the MVPA data
        speed_corr = resampled[' SPEED'].str.split('km/h',expand=True).drop(columns = [1]).astype(float)
        speed_corr.columns = [' SPEED']
        coordinates = pd.concat([resampled[[' LATITUDE', ' LONGITUDE']], speed_corr[' SPEED']], axis = 1)
        coordinates = coordinates.drop_duplicates()

        phy_act_coordinates = coordinates[(coordinates[' SPEED']>5) & (coordinates[' SPEED']<20)]
        still_coordinates = coordinates[(coordinates[' SPEED']<1)]


        resampled['MVPA'] = 0
        for ind in list(phy_act_coordinates.index.values):
          resampled["MVPA"][ind] = 1

        #Add the age and sex
        columns = ['age', 'Sexe', 'unique_id']
        resampled = resampled.merge(pData[columns], on='unique_id', how='left')
        
        #Add the NDVI data
        resampled = resampled.round(3)
        resampled[' UTC DATE']= pd.to_datetime(resampled[' UTC DATE'], format='%Y/%m/%d') 
        data = resampled.merge(NDVI_processed, on=[' UTC DATE',' LATITUDE',' LONGITUDE'] , how='left')
        data.dropna(subset=['value'])

        #Add the built environment factors
        columns = ['capacity', ' LATITUDE',' LONGITUDE']
        data = data.merge(transitStops[columns], on=[' LATITUDE',' LONGITUDE'], how='left')
        columns = ['tree', ' LATITUDE',' LONGITUDE']
        data = data.merge(trees[columns], on=[' LATITUDE',' LONGITUDE'], how='left')
    except:
        print('error', filename)
        return

    return data.fillna(0)

In [82]:
#combine all files in the list
combined_csv = pd.concat([preprocessing(f)for f in tqdm(all_filenames)])
#export to csv
combined_csv.to_csv( "goldData.csv", )

  0%|          | 0/273 [00:00<?, ?it/s]/Users/anurag/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/anurag/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/anurag/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

error goldData.csv


 52%|█████▏    | 142/273 [23:16<22:45, 10.43s/it]

error 2018-05-17_61_np3_1E_9002-A.csv


100%|██████████| 273/273 [45:08<00:00,  9.92s/it]


In [81]:
combined_csv

,INDEX,RCR,UTC DATE,UTC TIME,LOCAL DATE,LOCAL TIME,MS,VALID,LATITUDE,N/S,...,NSAT(USED/VIEW),file_name,unique_id,MVPA,age,Sexe,variable,value,capacity,tree
0,1,T,2018-05-22,07:03:02,2018/05/22,09:03:02,0,SPS,48.871,N,...,4/ 5,2018-05-19_66_np6_5G_10685-A.csv,10685-A,0,56.76,1,2442_48871,3841.0,0.0,0.0
1,13,T,2018-05-22,07:04:02,2018/05/22,09:04:02,0,SPS,48.871,N,...,4/ 5,2018-05-19_66_np6_5G_10685-A.csv,10685-A,0,56.76,1,2442_48871,3841.0,0.0,0.0
2,25,T,2018-05-22,07:05:02,2018/05/22,09:05:02,0,SPS,48.871,N,...,3/ 5,2018-05-19_66_np6_5G_10685-A.csv,10685-A,0,56.76,1,2442_48871,3841.0,0.0,0.0
3,37,T,2018-05-22,07:06:02,2018/05/22,09:06:02,0,SPS,48.871,N,...,4/ 6,2018-05-19_66_np6_5G_10685-A.csv,10685-A,0,56.76,1,2442_48871,3841.0,0.0,0.0
4,49,T,2018-05-22,07:07:02,2018/05/22,09:07:02,0,SPS,48.871,N,...,6/ 6,2018-05-19_66_np6_5G_10685-A.csv,10685-A,0,56.76,1,2442_48871,3841.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7238,76585,T,2019-07-24,06:27:49,2019/07/24,08:27:49,0,DGPS,48.850,N,...,7/10,2019-07-17_237_np184_2IQ_20416-B.csv,20416-B,0,47.63,2,2384_48850,1891.0,0.0,1.0
7239,76585,T,2019-07-24,06:27:49,2019/07/24,08:27:49,0,DGPS,48.850,N,...,7/10,2019-07-17_237_np184_2IQ_20416-B.csv,20416-B,0,47.63,2,2384_48850,1891.0,0.0,1.0
7240,76585,T,2019-07-24,06:27:49,2019/07/24,08:27:49,0,DGPS,48.850,N,...,7/10,2019-07-17_237_np184_2IQ_20416-B.csv,20416-B,0,47.63,2,2384_48850,1891.0,0.0,1.0
7241,76585,T,2019-07-24,06:27:49,2019/07/24,08:27:49,0,DGPS,48.850,N,...,7/10,2019-07-17_237_np184_2IQ_20416-B.csv,20416-B,0,47.63,2,2384_48850,1891.0,0.0,1.0
